In [1]:
import pandas as pd
import numpy as np
from fyers_apiv3 import fyersModel
import tensorflow as tf
from indicators import alligator, vroc, sma, epoch_to_time, time_to_epoch
import pandas as pd
import numpy as np
import time, datetime
client_id = "ZHQ4IJL7TI-100"
model = tf.keras.models.load_model("07OCT.keras")
with open("access_token", "r") as f:
    access_token = f.read()

fyers = fyersModel.FyersModel(client_id=client_id, is_async=False, token=access_token, log_path="")

In [2]:
data = pd.read_csv('data/zeel/NSE-ZEEL-EQ_4.csv')
data

,DATE,OPEN,HIGH,LOW,CLOSE,JAW,TEETH,LIPS,MA5,MA15,VOLUME,VROC
0,'2024-07-01 09:48:00',152.08,152.12,152.00,152.00,151.67,151.46,151.96,152.08,151.84,41178,154.34
1,'2024-07-01 09:49:00',152.00,152.01,151.95,152.01,151.65,151.47,151.98,152.06,151.88,6299,-93.42
2,'2024-07-01 09:50:00',152.00,152.00,151.87,152.00,151.63,151.49,151.98,152.02,151.93,15102,-62.87
3,'2024-07-01 09:51:00',152.00,152.05,151.94,152.00,151.61,151.51,151.97,152.00,151.95,14581,161.92
4,'2024-07-01 09:52:00',152.05,152.15,152.00,152.03,151.60,151.54,151.98,152.01,151.98,23172,-41.08
...,...,...,...,...,...,...,...,...,...,...,...,...
25461,'2024-10-07 15:24:00',126.17,126.25,125.91,125.94,126.33,126.29,125.99,126.11,125.86,133939,-7.16
25462,'2024-10-07 15:25:00',126.00,126.00,125.84,125.90,126.33,126.21,126.01,126.09,125.86,112741,-17.70
25463,'2024-10-07 15:26:00',125.90,125.95,125.80,125.94,126.34,126.15,125.99,126.05,125.87,43491,-18.80
25464,'2024-10-07 15:27:00',125.95,125.95,125.90,125.95,126.34,126.08,125.97,126.00,125.88,57202,-11.34


In [3]:
data = data.to_numpy()

In [4]:
date_today = datetime.datetime.now().strftime("%Y-%m-%d")
date_today
last_date = data[-1, 0].split(" ")[0][1:]
date_obj = datetime.datetime.strptime(last_date, '%Y-%m-%d')
new_date_obj = date_obj + datetime.timedelta(days=1)
last_date_1 = new_date_obj.strftime('%Y-%m-%d')

last_date, last_date_1, date_today

('2024-10-07', '2024-10-08', '2024-10-10')

In [5]:
data[-1]

array(["'2024-10-07 15:28:00'", 125.95, 126.25, 125.91, 126.15, 126.35,
       126.03, 125.96, 125.98, 125.91, 69203, -17.21], dtype=object)

In [6]:
response = fyers.history(data={"symbol": "NSE:ZEEL-EQ",
                                "resolution": "1",
                                "date_format": "1",
                                "range_from": last_date_1,
                                "range_to": date_today,
                                "cont_flag": "1"
                                })
len(response['candles']), response

(1034,
 {'candles': [[1728359100, 126.05, 127.35, 126.05, 126.6, 165283],
   [1728359160, 126.6, 126.6, 125.91, 126.48, 148542],
   [1728359220, 126.44, 126.57, 125.73, 125.95, 104263],
   [1728359280, 125.85, 125.9, 125.61, 125.84, 60141],
   [1728359340, 125.85, 125.87, 125.5, 125.67, 73525],
   [1728359400, 125.67, 125.9, 125.23, 125.9, 92971],
   [1728359460, 126, 126.64, 125.9, 126.55, 134848],
   [1728359520, 126.55, 126.73, 126.46, 126.58, 75896],
   [1728359580, 126.6, 126.8, 126.55, 126.66, 60227],
   [1728359640, 126.75, 126.88, 126.57, 126.88, 62956],
   [1728359700, 126.88, 127.18, 126.75, 127.02, 69092],
   [1728359760, 127.02, 127.02, 126.37, 126.8, 55831],
   [1728359820, 126.8, 127.05, 126.7, 127, 58762],
   [1728359880, 127, 127.15, 126.82, 127, 48402],
   [1728359940, 127, 127.8, 126.95, 127.56, 99746],
   [1728360000, 127.56, 127.7, 127.35, 127.39, 67115],
   [1728360060, 127.39, 127.53, 127.23, 127.53, 60523],
   [1728360120, 127.53, 127.7, 127.41, 127.45, 51182],
 

In [7]:
cleaned_reasponse = []
for i in response['candles']:
    z = 0
    for j in response['candles']:
        if i[0] == j[0]:
            z += 1
    if z > 1:
        print("repeted: ", i)
    else:
        cleaned_reasponse.append(i)
cleaned_reasponse

[[1728359100, 126.05, 127.35, 126.05, 126.6, 165283],
 [1728359160, 126.6, 126.6, 125.91, 126.48, 148542],
 [1728359220, 126.44, 126.57, 125.73, 125.95, 104263],
 [1728359280, 125.85, 125.9, 125.61, 125.84, 60141],
 [1728359340, 125.85, 125.87, 125.5, 125.67, 73525],
 [1728359400, 125.67, 125.9, 125.23, 125.9, 92971],
 [1728359460, 126, 126.64, 125.9, 126.55, 134848],
 [1728359520, 126.55, 126.73, 126.46, 126.58, 75896],
 [1728359580, 126.6, 126.8, 126.55, 126.66, 60227],
 [1728359640, 126.75, 126.88, 126.57, 126.88, 62956],
 [1728359700, 126.88, 127.18, 126.75, 127.02, 69092],
 [1728359760, 127.02, 127.02, 126.37, 126.8, 55831],
 [1728359820, 126.8, 127.05, 126.7, 127, 58762],
 [1728359880, 127, 127.15, 126.82, 127, 48402],
 [1728359940, 127, 127.8, 126.95, 127.56, 99746],
 [1728360000, 127.56, 127.7, 127.35, 127.39, 67115],
 [1728360060, 127.39, 127.53, 127.23, 127.53, 60523],
 [1728360120, 127.53, 127.7, 127.41, 127.45, 51182],
 [1728360180, 127.45, 127.69, 127.41, 127.69, 120868],


In [8]:
prev_ref_data = data[-375:, :11].tolist()
len(prev_ref_data), prev_ref_data

(375,
 [["'2024-10-04 15:29:00'",
   131.98,
   132.2,
   131.98,
   132.0,
   132.3,
   131.89,
   131.92,
   131.99,
   131.9,
   95036],
  ["'2024-10-07 09:15:00'",
   132.6,
   133.26,
   132.12,
   132.69,
   132.26,
   131.86,
   131.96,
   132.12,
   131.96,
   263098],
  ["'2024-10-07 09:16:00'",
   132.8,
   133.27,
   132.65,
   133.1,
   132.23,
   131.83,
   132.1,
   132.35,
   132.05,
   101220],
  ["'2024-10-07 09:17:00'",
   133.1,
   133.14,
   132.84,
   133.1,
   132.2,
   131.82,
   132.27,
   132.58,
   132.15,
   90965],
  ["'2024-10-07 09:18:00'",
   133.1,
   133.1,
   132.75,
   132.84,
   132.18,
   131.8,
   132.42,
   132.75,
   132.23,
   97592],
  ["'2024-10-07 09:19:00'",
   132.84,
   132.85,
   132.5,
   132.73,
   132.15,
   131.82,
   132.52,
   132.89,
   132.29,
   90469],
  ["'2024-10-07 09:20:00'",
   132.67,
   132.72,
   132.35,
   132.45,
   132.13,
   131.87,
   132.55,
   132.84,
   132.34,
   77457],
  ["'2024-10-07 09:21:00'",
   132.45,
  

In [9]:
prev_ref_data_clean = []
for indx, p in enumerate(prev_ref_data):
    date_obj = datetime.datetime.strptime(p[0], "'%Y-%m-%d %H:%M:%S'")
    epoch_time = int(time.mktime(date_obj.timetuple()))
    # prev_ref_data[indx][0] = epoch_time
    prev_ref_data_clean.append(
        [epoch_time] + p[1:5] + [p[10 ]]
    )
len(prev_ref_data_clean), prev_ref_data_clean

(375,
 [[1728035940, 131.98, 132.2, 131.98, 132.0, 95036],
  [1728272700, 132.6, 133.26, 132.12, 132.69, 263098],
  [1728272760, 132.8, 133.27, 132.65, 133.1, 101220],
  [1728272820, 133.1, 133.14, 132.84, 133.1, 90965],
  [1728272880, 133.1, 133.1, 132.75, 132.84, 97592],
  [1728272940, 132.84, 132.85, 132.5, 132.73, 90469],
  [1728273000, 132.67, 132.72, 132.35, 132.45, 77457],
  [1728273060, 132.45, 132.65, 132.33, 132.65, 55183],
  [1728273120, 132.65, 132.65, 132.43, 132.55, 41520],
  [1728273180, 132.45, 132.49, 132.0, 132.2, 172895],
  [1728273240, 132.2, 132.29, 132.01, 132.01, 50415],
  [1728273300, 132.02, 132.29, 131.86, 131.89, 88761],
  [1728273360, 131.88, 131.9, 131.44, 131.53, 78935],
  [1728273420, 131.61, 131.85, 131.57, 131.85, 30375],
  [1728273480, 131.8, 131.88, 131.7, 131.88, 24723],
  [1728273540, 131.81, 131.89, 131.55, 131.74, 46100],
  [1728273600, 131.73, 131.8, 131.61, 131.61, 24920],
  [1728273660, 131.6, 131.67, 131.41, 131.44, 103425],
  [1728273720, 131

In [10]:
data_3_now = prev_ref_data_clean + cleaned_reasponse
len(data_3_now), data_3_now

(1409,
 [[1728035940, 131.98, 132.2, 131.98, 132.0, 95036],
  [1728272700, 132.6, 133.26, 132.12, 132.69, 263098],
  [1728272760, 132.8, 133.27, 132.65, 133.1, 101220],
  [1728272820, 133.1, 133.14, 132.84, 133.1, 90965],
  [1728272880, 133.1, 133.1, 132.75, 132.84, 97592],
  [1728272940, 132.84, 132.85, 132.5, 132.73, 90469],
  [1728273000, 132.67, 132.72, 132.35, 132.45, 77457],
  [1728273060, 132.45, 132.65, 132.33, 132.65, 55183],
  [1728273120, 132.65, 132.65, 132.43, 132.55, 41520],
  [1728273180, 132.45, 132.49, 132.0, 132.2, 172895],
  [1728273240, 132.2, 132.29, 132.01, 132.01, 50415],
  [1728273300, 132.02, 132.29, 131.86, 131.89, 88761],
  [1728273360, 131.88, 131.9, 131.44, 131.53, 78935],
  [1728273420, 131.61, 131.85, 131.57, 131.85, 30375],
  [1728273480, 131.8, 131.88, 131.7, 131.88, 24723],
  [1728273540, 131.81, 131.89, 131.55, 131.74, 46100],
  [1728273600, 131.73, 131.8, 131.61, 131.61, 24920],
  [1728273660, 131.6, 131.67, 131.41, 131.44, 103425],
  [1728273720, 13

In [11]:
np.array(data_3_now)

array([[1.72803594e+09, 1.31980000e+02, 1.32200000e+02, 1.31980000e+02,
        1.32000000e+02, 9.50360000e+04],
       [1.72827270e+09, 1.32600000e+02, 1.33260000e+02, 1.32120000e+02,
        1.32690000e+02, 2.63098000e+05],
       [1.72827276e+09, 1.32800000e+02, 1.33270000e+02, 1.32650000e+02,
        1.33100000e+02, 1.01220000e+05],
       ...,
       [1.72854876e+09, 1.29820000e+02, 1.29890000e+02, 1.29820000e+02,
        1.29830000e+02, 1.69300000e+03],
       [1.72854882e+09, 1.29890000e+02, 1.29890000e+02, 1.29820000e+02,
        1.29830000e+02, 1.02400000e+03],
       [1.72854888e+09, 1.29820000e+02, 1.29880000e+02, 1.29820000e+02,
        1.29880000e+02, 6.80000000e+02]])

In [12]:
for i in data_3_now:
    z = 0
    for j in data_3_now:
        if i[0] == j[0]:
            if i == j:
                pass
            else: print("same same but different")
            z += 1
    if z > 1:
        print(i)
        data_3_now.remove(i)

In [13]:
temp = []
for c in data_3_now:
    temp.append(sum(c[2:4])/2)

In [14]:
alli = alligator(pd.Series(temp))
alli = alli.to_numpy()
alli

array([[         nan,          nan,          nan],
       [         nan,          nan,   0.        ],
       [         nan,          nan,   0.        ],
       ...,
       [129.9002212 , 129.84213991, 129.84746774],
       [129.89520418, 129.83999742, 129.84897419],
       [129.89095771, 129.84249775, 129.85017935]])

In [15]:
dates = []
X = []
k = 0
for i in range(len(data_3_now)-1):
# for i in range(10):
    candle = data_3_now[i]
    d = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(candle[0]))
    if d not in dates:
        dates.append(d)
        temp = []
        temp.append(d)
        temp.append(candle[1])
        temp.append(candle[2])
        temp.append(candle[3])
        temp.append(candle[4])
        temp.append(round(alli[i][0], 2))
        temp.append(round(alli[i][1], 2))
        temp.append(round(alli[i][2], 2))
        temp.append(round(sma(np.array(data_3_now)[:i+1, 4], 5)[-1], 2))
        temp.append(round(sma(np.array(data_3_now)[:i+1, 4], 15)[-1], 2))
        temp.append(round(vroc(np.array(data_3_now)[:i+1, 5], 14), 2))
        X.append(temp)
        print("id:", i, "\tdata:", temp)
    else:
        k += 1
        print("skipping", k, "\t", d)

id: 0 	data: ['2024-10-04 15:29:00', 131.98, 132.2, 131.98, 132.0, nan, nan, nan, nan, nan, nan]
id: 1 	data: ['2024-10-07 09:15:00', 132.6, 133.26, 132.12, 132.69, nan, nan, 0.0, nan, nan, nan]
id: 2 	data: ['2024-10-07 09:16:00', 132.8, 133.27, 132.65, 133.1, nan, nan, 0.0, nan, nan, nan]
id: 3 	data: ['2024-10-07 09:17:00', 133.1, 133.14, 132.84, 133.1, nan, nan, 0.0, nan, nan, nan]
id: 4 	data: ['2024-10-07 09:18:00', 133.1, 133.1, 132.75, 132.84, nan, nan, 0.0, 132.75, nan, nan]
id: 5 	data: ['2024-10-07 09:19:00', 132.84, 132.85, 132.5, 132.73, nan, nan, 132.73, 132.89, nan, nan]
id: 6 	data: ['2024-10-07 09:20:00', 132.67, 132.72, 132.35, 132.45, nan, nan, 132.72, 132.84, nan, nan]
id: 7 	data: ['2024-10-07 09:21:00', 132.45, 132.65, 132.33, 132.65, nan, nan, 132.68, 132.75, nan, nan]
id: 8 	data: ['2024-10-07 09:22:00', 132.65, 132.65, 132.43, 132.55, nan, nan, 132.64, 132.64, nan, nan]
id: 9 	data: ['2024-10-07 09:23:00', 132.45, 132.49, 132.0, 132.2, nan, nan, 132.62, 132.52,

In [16]:
X = np.array(X)
X

array([['2024-10-04 15:29:00', '131.98', '132.2', ..., 'nan', 'nan',
        'nan'],
       ['2024-10-07 09:15:00', '132.6', '133.26', ..., 'nan', 'nan',
        'nan'],
       ['2024-10-07 09:16:00', '132.8', '133.27', ..., 'nan', 'nan',
        'nan'],
       ...,
       ['2024-10-10 13:55:00', '129.87', '129.87', ..., '129.84',
        '129.85', '-19.81'],
       ['2024-10-10 13:56:00', '129.82', '129.89', ..., '129.84',
        '129.84', '-64.82'],
       ['2024-10-10 13:57:00', '129.89', '129.89', ..., '129.83',
        '129.84', '-62.86']], dtype='<U32')

In [17]:
X[-1], X.shape

(array(['2024-10-10 13:57:00', '129.89', '129.89', '129.82', '129.83',
        '129.9', '129.84', '129.85', '129.83', '129.84', '-62.86'],
       dtype='<U32'),
 (1408, 11))

In [18]:
X_test = []
X_temp = np.array(X[-9, 1:], dtype=np.float32)
first_open = X_temp[0]
first_jaw = X_temp[4]
first_lip = X_temp[6]
first_teeth = X_temp[5]
first_ma5 = X_temp[7]
first_ma15 = X_temp[8]
first_volumn_change = X_temp[9]
for candle in np.array(X[-8:, 1:], dtype=np.float32):
    X_test.append([
        candle[0] - first_open,
        candle[1] - first_open,
        candle[2] - first_open,
        candle[3] - first_open,
        candle[4] - first_jaw,
        candle[5] - first_teeth,
        candle[6] - first_lip,
        candle[7] - first_ma5,
        candle[8] - first_ma15,
        candle[9] - first_volumn_change
        
    ])
X_test = np.array([X_test])
X_test.shape, X_test

((1, 8, 10),
 array([[[-9.9945068e-03, -9.9945068e-03, -7.9986572e-02, -3.9993286e-02,
          -2.0004272e-02,  0.0000000e+00,  0.0000000e+00,  1.0009766e-02,
           0.0000000e+00, -1.9899979e+00],
         [ 2.0004272e-02,  2.0004272e-02, -9.9990845e-02, -7.9986572e-02,
          -2.9998779e-02, -1.0009766e-02,  0.0000000e+00,  0.0000000e+00,
          -1.0009766e-02,  3.0534000e+02],
         [-2.9998779e-02, -9.9945068e-03, -8.9996338e-02, -9.9945068e-03,
          -3.9993286e-02, -2.0004272e-02,  0.0000000e+00,  2.0004272e-02,
           0.0000000e+00,  1.1261000e+02],
         [-1.6999817e-01, -1.9989014e-02, -1.6999817e-01, -9.9990845e-02,
          -5.0003052e-02, -1.0009766e-02,  0.0000000e+00,  0.0000000e+00,
           0.0000000e+00,  9.1870003e+01],
         [-1.4999390e-01,  9.0011597e-02, -1.4999390e-01, -6.9992065e-02,
          -5.0003052e-02, -1.0009766e-02, -9.9945068e-03,  0.0000000e+00,
           0.0000000e+00,  7.7597003e+02],
         [-2.9998779e-02, -2.999

In [19]:
pred = model.predict(X_test)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step


array([[-0.02838728, -0.0492444 , -0.03594962]], dtype=float32)

In [20]:
{"DATETIME": X[-1][0],
"Org": X[-1][1],
"C1: ": round(float(X[-1, 1]) + pred[0, 0], 2),
"C2: ": round(float(X[-1, 2]) + pred[0, 1], 2),
"C3: ": round(float(X[-1, 3]) + pred[0, 2], 2)}

{'DATETIME': '2024-10-10 13:57:00',
 'Org': '129.89',
 'C1: ': 129.86,
 'C2: ': 129.84,
 'C3: ': 129.78}